In [2]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from tqdm import tqdm


In [3]:
# Step 1: Data Preparation
class DepressionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [4]:
# Load Dataset
data_path = "C:/Users/starf/OneDrive/Bureau/depression1.zip"
dataset = pd.read_csv(data_path)

In [8]:
# Extract texts and labels
texts = dataset['clean_text'].tolist()
labels = dataset['is_depression'].tolist()

In [10]:
# Split Data
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

In [20]:
# Initialize Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [22]:
# Create Datasets
train_dataset = DepressionDataset(train_texts, train_labels, tokenizer, max_length=128)
val_dataset = DepressionDataset(val_texts, val_labels, tokenizer, max_length=128)

In [24]:
# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)


In [18]:

# Step 2: Model Setup
# Load the pretrained BERT model for sequence classification
# Setting num_labels=2 for binary classification
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased', 
    num_labels=2
)

# Check if GPU (CUDA) is available, else default to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the specified device
model.to(device)

# Print confirmation of the device being used
print(f"Model is set up on device: {device}")


model.safetensors:  79%|#######8  | 346M/440M [00:00<?, ?B/s]

C:\Users\starf\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\starf\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly

Model is set up on device: cpu


In [28]:
# Step 3: Training
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    loop = tqdm(train_loader, leave=True)
    total_loss = 0

    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        loop.set_description(f'Epoch {epoch + 1}')
        loop.set_postfix(loss=loss.item())

    print(f"Epoch {epoch + 1} Loss: {total_loss / len(train_loader)}")







Epoch 1: 100%|█████████████████████████████████████████████████████████| 773/773 [34:28<00:00,  2.68s/it, loss=0.00114]


Epoch 1 Loss: 0.10830971926440489


Epoch 2:   1%|▎                                                          | 4/773 [00:13<41:40,  3.25s/it, loss=0.00955]


KeyboardInterrupt: 

In [30]:
# Step 4: Evaluation
model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, axis=1)

        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())


In [32]:
# Classification Report
print(classification_report(true_labels, predictions, target_names=['Not Depressed', 'Depressed']))

               precision    recall  f1-score   support

Not Depressed       0.97      0.98      0.98       783
    Depressed       0.98      0.97      0.97       764

     accuracy                           0.98      1547
    macro avg       0.98      0.98      0.98      1547
 weighted avg       0.98      0.98      0.98      1547



In [34]:
# Step 5: Making Predictions
def predict(text, model, tokenizer, max_length=128):
    model.eval()
    encoding = tokenizer(
        text,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors="pt"
    )
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, axis=1)

    return "Depressed" if preds.item() == 1 else "Not Depressed"


In [64]:
# Test the Prediction Function
sample_text = "I feel like killing myself."
print(f"Prediction: {predict(sample_text, model, tokenizer)}")

Prediction: Depressed
